# طبقه‌بندی ساده تصاویر

این دفترچه به شما نشان می‌دهد که چگونه تصاویر را با استفاده از یک شبکه عصبی از پیش آموزش‌دیده طبقه‌بندی کنید.

**آنچه یاد خواهید گرفت:**
- نحوه بارگذاری و استفاده از یک مدل از پیش آموزش‌دیده
- پیش‌پردازش تصاویر
- انجام پیش‌بینی روی تصاویر
- درک امتیازات اعتماد به نفس

**مورد استفاده:** شناسایی اشیاء در تصاویر (مانند "گربه"، "سگ"، "ماشین" و غیره)

---


## مرحله ۱: وارد کردن کتابخانه‌های مورد نیاز

بیایید ابزارهایی که لازم داریم را وارد کنیم. نگران نباشید اگر هنوز همه این‌ها را متوجه نمی‌شوید!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## مرحله ۲: بارگذاری مدل از پیش آموزش‌دیده

ما از **MobileNetV2** استفاده خواهیم کرد، یک شبکه عصبی که قبلاً روی میلیون‌ها تصویر آموزش داده شده است.

این روش **یادگیری انتقالی** نامیده می‌شود - استفاده از مدلی که شخص دیگری آن را آموزش داده است!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## مرحله ۳: توابع کمکی

بیایید توابعی برای بارگذاری و آماده‌سازی تصاویر برای مدل خود ایجاد کنیم.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## مرحله ۴: آزمایش روی تصاویر نمونه

بیایید چند تصویر از اینترنت را طبقه‌بندی کنیم!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### طبقه‌بندی هر تصویر


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## مرحله ۵: تصاویر خود را امتحان کنید!

آدرس URL زیر را با هر آدرس URL تصویری که می‌خواهید طبقه‌بندی کنید، جایگزین کنید.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 چه اتفاقی افتاد؟

1. **ما یک مدل از پیش آموزش‌دیده را بارگذاری کردیم** - MobileNetV2 روی میلیون‌ها تصویر آموزش داده شده است  
2. **تصاویر را پیش‌پردازش کردیم** - آن‌ها را تغییر اندازه داده و برای مدل آماده کردیم  
3. **مدل پیش‌بینی انجام داد** - احتمال‌ها را برای 1000 کلاس شیء خروجی داد  
4. **نتایج را رمزگشایی کردیم** - اعداد را به برچسب‌های قابل فهم برای انسان تبدیل کردیم  

### درک امتیازات اعتماد

- **90-100%**: بسیار مطمئن (تقریباً قطعاً درست است)  
- **70-90%**: مطمئن (احتمالاً درست است)  
- **50-70%**: تا حدی مطمئن (ممکن است درست باشد)  
- **کمتر از 50%**: خیلی مطمئن نیست (نامطمئن)  

### چرا ممکن است پیش‌بینی‌ها اشتباه باشند؟

- **زاویه یا نور غیرمعمول** - مدل روی عکس‌های معمولی آموزش دیده است  
- **اشیاء متعدد** - مدل انتظار یک شیء اصلی را دارد  
- **اشیاء نادر** - مدل فقط 1000 دسته را می‌شناسد  
- **تصویر با کیفیت پایین** - تصاویر تار یا پیکسلی تشخیص را سخت‌تر می‌کنند  

---


## 🚀 مراحل بعدی

1. **تصاویر مختلف را امتحان کنید:**
   - تصاویر را در [Unsplash](https://unsplash.com) پیدا کنید
   - کلیک راست کنید → "Copy image address" برای دریافت URL

2. **آزمایش کنید:**
   - با هنر انتزاعی چه اتفاقی می‌افتد؟
   - آیا می‌تواند اشیاء را از زوایای مختلف تشخیص دهد؟
   - چگونه با اشیاء متعدد برخورد می‌کند؟

3. **بیشتر یاد بگیرید:**
   - [درس‌های بینایی کامپیوتر](../lessons/4-ComputerVision/README.md) را بررسی کنید
   - یاد بگیرید چگونه یک طبقه‌بند تصویر خودتان را آموزش دهید
   - بفهمید شبکه‌های عصبی پیچشی (CNNs) چگونه کار می‌کنند

---

## 🎉 تبریک!

شما به تازگی یک طبقه‌بند تصویر با استفاده از یک شبکه عصبی پیشرفته ساختید!

همین تکنیک قدرت‌بخش موارد زیر است:
- Google Photos (سازماندهی عکس‌های شما)
- خودروهای خودران (تشخیص اشیاء)
- تشخیص پزشکی (تحلیل تصاویر اشعه ایکس)
- کنترل کیفیت (شناسایی نقص‌ها)

به یادگیری و کشف ادامه دهید! 🚀



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما هیچ مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
